<img style="float: left" src="images/spark.png" />
<img style="float: right" src="images/surfsara.png" />
<hr style="clear: both" />

# Introduction to Apache Spark

In this final notebook we will explore Spark using the Python API, called PySpark.

_You can edit the cells below and execute the code by selecting the cell and press Shift-Enter. Code completion is supported by use of the Tab key._

During the exercises you may want to refer to the [PySpark documentation](https://spark.apache.org/docs/1.6.0/api/python/pyspark.html#pyspark.RDD) for more information on possible transformations and actions.

In [1]:
# initialize Spark
from pyspark import SparkContext, SparkConf

if not 'sc' in globals(): # This 'trick' makes sure the SparkContext sc is initialized exactly once
    conf = SparkConf().setMaster('local[*]')
    sc = SparkContext(conf=conf)

## Creating an RDD from a list

All parallel work in Spark is done on RDDs, so the first thing we need to do is convert our data (in this case a list) to an RDD. We will use the `parallelize` method on the `SparkContext sc`. It takes two arguments: (1) the collection, and (2) the number of partitions (splits). The second argument is optional.

In [2]:
words_list = ['Dog', 'Cat', 'Rabbit', 'Hare', 'Deer', 'Gull', 'Woodpecker', 'Mole']
words_rdd = sc.parallelize(words_list, 2)

print 'the type of words_list is: ' + str(type(words_list))
print 'the type of words_rdd is: ' + str(type(words_rdd))

the type of words_list is: <type 'list'>
the type of words_rdd is: <class 'pyspark.rdd.RDD'>


## Map transformation

We now want to change all words in the `words_rdd` to their plural form. We will do this using a [map](https://spark.apache.org/docs/1.6.0/api/python/pyspark.html?highlight=map#pyspark.RDD.map) transformation.
Remember that the map function will apply the function to each element of the RDD.

First, we will write a simple function that takes a single word as argument and return the word with an 's' added to it. In the next step we will use this function in a map transformation of the `words_rdd`.

In [3]:
# TODO: Replace <FILL IN> with appropriate code
def make_plural(word):
    """Adds an 's' to `word`.

    Note:
        This is a simple function that only adds an 's'.  

    Args:
        word (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    return word + 's'

print make_plural('cat')

cats


Next, we will use the `makePlural` function as input for the map transformation on `words_rdd`.
The action [`collect`](https://spark.apache.org/docs/1.6.0/api/python/pyspark.html?highlight=collect#pyspark.RDD.collect) transfers the content of the RDD to the driver. Note, that a large RDD may be scattered over many machines. In such a case a `collect` can be a very bad idea.

In [4]:
# TODO: Replace <FILL IN> with appropriate code
plural_rdd = words_rdd.map(make_plural)
print plural_rdd.collect()

['Dogs', 'Cats', 'Rabbits', 'Hares', 'Deers', 'Gulls', 'Woodpeckers', 'Moles']


## Using lambda functions

But wait a minute! We can actually achieve the same functionality by using lambda functions. In this case we define makePlural as a lambda function. In this case we define `makePlural` as a lambda function.

Hint: The map function needs a function as argument. This function needs one argument, let's call that `x`. The body of the function adds an 's' to the end of `x`.

In [5]:
# TODO: Replace <FILL IN> with appropriate code

# A lambda function for adding s at the end of a string
lambda_plural_rdd = words_rdd.map(lambda x: x + 's')

print lambda_plural_rdd.collect()

['Dogs', 'Cats', 'Rabbits', 'Hares', 'Deers', 'Gulls', 'Woodpeckers', 'Moles']


Let's do another map transformation. For each word in `words_rdd` determine its length. The Python function `len`  will return the length of a string.

You can do this with a lambda function, but there is another way.

In [7]:
# TODO: Replace <FILL IN> with appropriate code
word_lengths = (words_rdd.map(len).collect())
print word_lengths

[3, 3, 6, 4, 4, 4, 10, 4]


Test your solution by running the following cell:

In [8]:
from test_helper import Test

Test.assertEquals(sorted(word_lengths), [3, 3, 4, 4, 4, 4, 6, 10], 'incorrect value for word_lengths')

1 test passed.


 ## RDD from a text file

Manipulating a list with eight elements gets boring pretty fast, so let's move start processing a file. In this part we will read the file `alice.txt` and will count the number of occurrences of every word. This 'Word Count' is the ['Hello World'](https://en.wikipedia.org/wiki/%22Hello,_World!%22_program) of data-processing frameworks.

In [9]:
# create an RDD from alice.txt where every element is a line of the file
alice_rdd = sc.textFile('file://///home/jovyan/work/data/alice.txt').cache()

## `collect` call not accepted

As we mentioned before, once your data no longer fits on the screen the `collect` method becomes less useful or even problematic. But we still want to have a way to inspect the (intermediate) results. For this we can use one of the following methods as a replacement of `collect`:

- `first()`: return the first elements of the RDD
- `take(n)`:  return a list of `n` elements
- `takeOrdered(n, [key=f])`: return the first `n` elements of the RDD, the order is defined by the optional function `f`.

In [10]:
import pprint
pp = pprint.PrettyPrinter(indent=2)

print 'first: ' + alice_rdd.first()
print 'take(5): '
pp.pprint(alice_rdd.take(5))

first: The Project Gutenberg EBook of Alice in Wonderland, by Lewis Carroll
take(5): 
[ u'The Project Gutenberg EBook of Alice in Wonderland, by Lewis Carroll',
  u'',
  u'This eBook is for the use of anyone anywhere at no cost and with almost no',
  u'restrictions whatsoever. You may copy it, give it away or re-use it under the',
  u'terms of the Project Gutenberg License online at www.gutenberg.org']


## The one-to-many map: flatMap

We have an RDD of lines, so let's try to convert this to an RDD of words by splitting the lines on whitespace:

In [11]:
alice_words_try = alice_rdd.map(lambda line: line.split())

pp.pprint(alice_words_try.take(4))

[ [ u'The',
    u'Project',
    u'Gutenberg',
    u'EBook',
    u'of',
    u'Alice',
    u'in',
    u'Wonderland,',
    u'by',
    u'Lewis',
    u'Carroll'],
  [],
  [ u'This',
    u'eBook',
    u'is',
    u'for',
    u'the',
    u'use',
    u'of',
    u'anyone',
    u'anywhere',
    u'at',
    u'no',
    u'cost',
    u'and',
    u'with',
    u'almost',
    u'no'],
  [ u'restrictions',
    u'whatsoever.',
    u'You',
    u'may',
    u'copy',
    u'it,',
    u'give',
    u'it',
    u'away',
    u'or',
    u're-use',
    u'it',
    u'under',
    u'the']]


This doesn't look right! We want an RDD of words, but we have created an RDD of lists of words.

We want to map a function on an input that returns multiple values in a list, but then not to want the output nested in the same way as the input was. As it is commonly needed Spark includes a [`flatMap`](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=map#pyspark.RDD.flatMap) transformation that will _flatten_ the output of the function.

In [12]:
alice_words = alice_rdd.flatMap(lambda line: line.split())

pp.pprint(alice_words.take(4))

[u'The', u'Project', u'Gutenberg', u'EBook']


## Key-Value Pairs

In order to count words in parallel we are going to use an RDD which consist of simple key-value pairs. We will call this RDD `alice_pairs` and it will be result of a transformation of `alice_words`. For every word in `alice_words` we want to have a `(word, 1)` tuple.

In [13]:
# TODO: Replace <FILL IN> with appropriate code
alice_pairs = alice_words.map(lambda x: (x, 1))

print alice_pairs.takeOrdered(10)

[(u'"\'--found', 1), (u'"\'Tis', 1), (u'"--\'And', 1), (u'"A', 1), (u'"A', 1), (u'"Ah,', 1), (u'"Ahem!"', 1), (u'"Alice!"', 1), (u'"And', 1), (u'"And', 1)]


In [14]:
Test.assertEquals(alice_pairs.takeOrdered(10),
                  [(u'"\'--found', 1), (u'"\'Tis', 1), (u'"--\'And', 1), (u'"A', 1), (u'"A', 1), (u'"Ah,', 1), (u'"Ahem!"', 1), (u'"Alice!"', 1), (u'"And', 1), (u'"And', 1)],
                  'incorrect value for alice_pairs')

1 test passed.


## ReduceByKey

Next, we are going to count all words by using [`reduceByKey`](https://spark.apache.org/docs/1.6.0/api/python/pyspark.html?highlight=reducebykey#pyspark.RDD.reduceByKey).

`ReducebyKey` expects the RDD to consist of key-value pairs an it will perform a reduce operation per key.
It will need a two-argument function as input that will work on the values only. Remember that a reduce function needs two arguments and will reduce all elements to a single value.

In [15]:
# TODO: Replace <FILL IN> with appropriate code

# Note that reduceByKey takes in a function that accepts two values and returns a single value
# The function that is input to reduceByKey only works on the values. Spark will execute this function per key

word_counts = alice_pairs.reduceByKey(lambda x,y : x + y)

top10_words = word_counts.takeOrdered(10, key=lambda p: -p[1])

print top10_words

[(u'the', 577), (u'and', 296), (u'a', 265), (u'to', 241), (u'she', 197), (u'of', 197), (u'was', 159), (u'in', 157), (u'said', 129), (u'it', 114)]


In [16]:
# TEST Counting using reduceByKey
Test.assertEquals(top10_words,
                  [(u'the', 577), (u'and', 296), (u'a', 265), (u'to', 241), (u'she', 197), (u'of', 197), (u'was', 159), (u'in', 157), (u'said', 129), (u'it', 114)]
,
                  'incorrect value for word_counts')

1 test passed.


## ReduceByKey - background

The `reduceByKey` method is similar to Hadoop's Reduce, but more restrictive. The function you provide to `reduceByKey` needs to be both [commutative](https://en.wikipedia.org/wiki/Commutative_property) and [associative](https://en.wikipedia.org/wiki/Associative_Property).

These restrictions allow Spark to perform additional optimisations, performing the operation on each partitions of the RDD and minimising network traffic. In the example below at most six values need to be transmitted.

![reduceByKey](images/reducebykey.png)
(Picture by DataBricks)